## Casamento de fase com biblioteca de meta-átomos (MP)
## Phase matching with meta-atom library (MP)

---

### Objetivo (PT)  
Este notebook executa a etapa de **casamento de fase (Phase Matching)** entre os mapas de fase-alvo gerados pelos algoritmos de **holografia (polarização X / TM)** e **grades de Dammann (polarização Y / TE)** e a **biblioteca numérica de meta-átomos** simulados em CST.  
A partir da biblioteca filtrada em $\lambda = 1064\ \text{nm}$, o código:
1. Carrega os mapas de fase produzidos pelo Gerchberg–Saxton (GS) para os alvos *ilum* e *ufabc*;
2. Executa o algoritmo de casamento de fase pixel a pixel, escolhendo o meta-átomo que minimiza o erro no plano complexo para TM e TE;
3. Reconstrói numericamente o holograma a partir do layout físico $(L_x, L_y)$ e calcula a correlação de Pearson com o alvo bruto;
4. Calcula métricas globais de **Eficiência de Difração (DE)** e **uniformidade (RMSE)** para o canal TE, reutilizando a mesma rotina de análise das grades de Dammann.

Esta implementação segue a lógica descrita na monografia e é compatível com a formulação de **Guo et al. (2025)** para metassuperfícies programáveis por polarização.

### Objective (EN)  
This notebook implements the **phase matching** step between the target phase maps generated by the **holography (X / TM polarization)** and **Dammann grating (Y / TE polarization)** algorithms and the **numerical meta-atom library** simulated in CST.  
Using the library filtered at $\lambda = 1064\ \text{nm}$, the code:
1. Loads the phase maps produced by the Gerchberg–Saxton (GS) algorithm for the *ilum* and *ufabc* targets;
2. Runs the phase-matching algorithm pixel by pixel, selecting the meta-atom that minimizes the complex-plane error for TM and TE;
3. Reconstructs the hologram numerically from the physical layout $(L_x, L_y)$ and computes the Pearson correlation with the raw target image;
4. Computes global **Diffraction Efficiency (DE)** and **uniformity (RMSE)** metrics for the TE channel, reusing the same analysis routine used for the Dammann gratings.

This implementation follows the logic described in the thesis and is consistent with the formulation of **Guo et al. (2025)** for polarization-programmable metasurfaces.

---

### 💾 Saídas / Outputs

| Tipo | Descrição | Local aproximado |
|------|-----------|------------------|
| `DataFrame` (Parquet / memória) | Biblioteca de meta-átomos filtrada em 1064 nm com campos complexos `S_complex_TE` e `S_complex_TM` | `data/meta_library/library_1064nm.parquet` e variável `library_df` |
| `npy` | Layout final dos semi-eixos $L_x$ e $L_y$ para cada imagem | `results/meta_library/phase_matching/demo_ilum_ufabc/<run_id>/<img>/layout_lx__<img>.npy` e `layout_ly__<img>.npy` |
| `npy` | Mapa de erro RMS por pixel (casamento de fase TM/TE) | `results/.../error_map__<img>.npy` |
| `csv` | Layouts $L_x$ e $L_y$ em formato tabular para fabricação / pós-processamento | `results/.../layout_lx__<img>.csv`, `layout_ly__<img>.csv` |
| `png` | Mapas de $L_x$, $L_y$ e erro RMS (fase matching) | `results/.../layouts_and_error__<img>.png` |
| `png` | Histograma da distribuição do erro RMS por pixel | `results/.../error_histogram__<img>.png` |
| `png` | Comparação alvo vs reconstrução holográfica (TM) | `results/.../reconstruction__<img>.png` |
| `txt` | Métricas numéricas por imagem: erro RMS médio e máximo, correlação de Pearson, DE e RMSE TE | `results/.../metrics__<img>.txt` |

---





### 1. Descoberta do repositório e configuração de paths  

**PT** — Esta célula:
- Localiza automaticamente a raiz do repositório (`repo_root`) subindo diretórios até encontrar a pasta `src`;
- Garante que `src` esteja em `sys.path`, permitindo importar módulos internos (`holography`, `meta_library`, etc.);
- Define as pastas principais usadas no notebook:
  - `bib_folder`: biblioteca de meta-átomos (arquivos Touchstone, CST);
  - `results_holo_root`: resultados das simulações de holografia (GS);
  - `results_pm_root`: resultados desta etapa de phase matching;
  - `targets_dir`: imagens-alvo brutas (como `ilum.png` e `ufabc.png`);
- Faz um *sanity check* para verificar se os arquivos de alvo bruto existem.

O *output* confirma que o diretório raiz foi detectado corretamente e que as imagens `ilum.png` e `ufabc.png` estão presentes e prontas para uso.

<details>
<summary><b>Show English</b></summary>

**EN** — This cell:
- Automatically locates the repository root (`repo_root`) by moving up directories until it finds `src`;
- Ensures that `src` is in `sys.path`, so internal modules (`holography`, `meta_library`, etc.) can be imported;
- Defines the main folders used in the notebook:
  - `bib_folder`: meta-atom library (Touchstone/CST files);
  - `results_holo_root`: holography (GS) simulation results;
  - `results_pm_root`: phase matching results;
  - `targets_dir`: raw target images (e.g., `ilum.png`, `ufabc.png`);
- Performs a quick sanity check to verify that the target images exist.

The *output* confirms that the root directory was correctly detected and both `ilum.png` and `ufabc.png` are available.



In [29]:
import sys
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

repo_root = Path.cwd()
while not (repo_root / "src").exists() and repo_root != repo_root.parent:
    repo_root = repo_root.parent

print("Repo root detectado:", repo_root)

src_path = repo_root / "src"
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))
    print("Adicionado ao sys.path:", src_path)

# Pastas principais
bib_folder = repo_root / "Bibliotecas" / "Bib1-27x27-perdas"
results_holo_root = repo_root / "results" / "holography"
results_pm_root = repo_root / "results" / "meta_library" / "phase_matching"

# Pasta com as imagens-alvo brutas (ilum.png, ufabc.png)
targets_dir = repo_root / "data" / "targets" / "common"
print("targets_dir:", targets_dir)

# (opcional) sanity check:
for name in ["ilum.png", "ufabc.png"]:
    p = targets_dir / name
    print(name, "existe?", p.exists())


Repo root detectado: c:\Users\vinicius23011\MATLAB\Projects\TCC
targets_dir: c:\Users\vinicius23011\MATLAB\Projects\TCC\data\targets\common
ilum.png existe? True
ufabc.png existe? True


#### Comentário

O print de repo_root e targets_dir ajuda a depurar problemas de path caso o notebook seja movido para outra máquina ou pasta.

### 2. Imports de módulos internos (meta_library + holography)  

**PT** — Esta célula importa os módulos internos que conectam o notebook ao restante do repositório:

- De `meta_library`:
  - `touchstone_to_dataframe`: converte a coleção de arquivos Touchstone (`.ts`) em um único `DataFrame` estruturado;
  - `append_derived_columns`: cria colunas derivadas, como os campos complexos `S_complex_TE` e `S_complex_TM`;
- De `holography`:
  - Importa o módulo `gs_asm` como `GSx`, que contém as rotinas de propagação ASM e reconstrução de imagens holográficas;
- O bloco `try/except` garante um *fallback*:
  - Primeiro tenta importar `gs_asm` como submódulo (`from holography import gs_asm`);
  - Caso falhe (por diferença de layout), importa direto `gs_asm` da raiz de `src`.

<details>
<summary><b>Show English</b></summary>

**EN** — This cell imports the internal modules that connect the notebook to the rest of the repo:

- From `meta_library`:
  - `touchstone_to_dataframe`: converts the Touchstone (`.ts`) collection into a single structured `DataFrame`;
  - `append_derived_columns`: builds derived columns such as complex fields `S_complex_TE` and `S_complex_TM`;
- From `holography`:
  - Imports `gs_asm` as `GSx`, which implements ASM propagation and holographic reconstruction;
- The `try/except` block provides a robust fallback:
  - First tries `from holography import gs_asm`;
  - If that fails, it imports `gs_asm` directly from `src`.


In [1]:
from meta_library.generate_df import touchstone_to_dataframe
from meta_library.clean_library import append_derived_columns

try:
    from holography import gs_asm as GSx
except ModuleNotFoundError:
    # fallback se o módulo estiver solto em src/
    import gs_asm as GSx


ModuleNotFoundError: No module named 'meta_library'

### 3. Funções utilitárias: busca de runs e carregamento de mapas de fase  

**PT** — Aqui são definidas três funções auxiliares importantes:

- `get_latest_subdir(base_dir)`  
  - Lista as subpastas de `base_dir`;
  - Ordena por nome e retorna a última (assumindo que as pastas de run seguem um padrão de timestamp, como `2025-11-16T03-37-41Z`);
  - Imprime qual foi o último run encontrado, útil para rastreabilidade dos experimentos.

- `load_phase_txt(path)`  
  - Carrega um mapa de fase salvo como `.txt` (geralmente via `np.savetxt`);
  - Aplica `np.mod(phase, 2π)` para normalizar as fases no intervalo $[0,2\pi)$, o que ajuda a manter a consistência entre diferentes etapas (GS, biblioteca, MP).

- `ensure_same_shape(phase_tm, phase_te)`  
  - Verifica se os mapas de fase TM (holograma, X) e TE (Dammann, Y) possuem o mesmo `shape`;
  - Caso contrário, lança um `ValueError` com mensagem clara;
  - Isso evita que o algoritmo de casamento de fase rode com matrizes incompatíveis, o que poderia causar erros silenciosos.

<details>
<summary><b>Show English</b></summary>

**EN** — This cell defines three important helper functions:

- `get_latest_subdir(base_dir)`  
  - Lists all subdirectories of `base_dir`;
  - Sorts them by name and returns the last one (assuming run folders follow a timestamp pattern like `2025-11-16T03-37-41Z`);
  - Prints which run was selected, which is handy for experiment traceability.

- `load_phase_txt(path)`  
  - Loads a phase map saved as `.txt` (typically via `np.savetxt`);
  - Applies `np.mod(phase, 2π)` to normalize phases in the $[0, 2\pi)$ range, keeping consistency across GS, library and MP steps.

- `ensure_same_shape(phase_tm, phase_te)`  
  - Checks whether TM (hologram, X) and TE (Dammann, Y) phase maps share the same `shape`;
  - If not, raises a `ValueError` with a clear message;
  - This prevents the phase-matching algorithm from running with mismatched matrices, which could lead to subtle bugs.


In [31]:
def get_latest_subdir(base_dir: Path) -> Path:
    """Retorna a subpasta mais recente (ordenado por nome) em base_dir."""
    subdirs = [d for d in base_dir.iterdir() if d.is_dir()]
    if not subdirs:
        raise FileNotFoundError(f"Nenhuma subpasta encontrada em {base_dir}")
    latest = sorted(subdirs)[-1]
    print(f"   Último run em {base_dir.name}: {latest.name}")
    return latest

def load_phase_txt(path: Path) -> np.ndarray:
    """Carrega mapa de fase salvo em .txt (np.savetxt)."""
    if not path.is_file():
        raise FileNotFoundError(f"Arquivo de fase não encontrado: {path}")
    phase = np.loadtxt(path)
    # Normaliza em [0, 2π) – não é obrigatório, mas ajuda a manter consistência
    return np.mod(phase, 2 * np.pi)

def ensure_same_shape(phase_tm: np.ndarray, phase_te: np.ndarray):
    """Garante que os mapas TM e TE tenham o mesmo shape."""
    if phase_tm.shape != phase_te.shape:
        raise ValueError(
            f"Shapes diferentes entre TM e TE: {phase_tm.shape} vs {phase_te.shape}"
        )
    return phase_tm, phase_te

### 4. Carregamento e cache da biblioteca de meta-átomos em 1064 nm  

**PT** — Esta célula prepara a **biblioteca de meta-átomos** usada no casamento de fase:

1. Define a frequência-alvo em GHz (`freq_target_ghz = 281760.0`), equivalente a $\lambda = 1064\ \text{nm}$;
2. Define o arquivo de cache Parquet (`library_1064nm.parquet`) em `data/meta_library/`;
3. Se o arquivo Parquet já existir:
   - Carrega o `DataFrame` em `df_1064`;
   - Se as colunas complexas não existirem (apenas real/imag), reconstrói `S_complex_TE` e `S_complex_TM` a partir de suas partes real e imaginária;
4. Se o cache não existir:
   - Usa `touchstone_to_dataframe` para converter todos os arquivos Touchstone da pasta `bib_folder` em um `DataFrame` agregado;
   - Filtra as linhas com `frequencia_ghz ≈ 281760.0`;
   - Usa `append_derived_columns` para criar as colunas complexas `S_complex_TE` (a partir de S13) e `S_complex_TM` (a partir de S24);
   - Decompõe essas colunas complexas em real/imag antes de salvar o Parquet, evitando problemas de tipo com complexos em Arrow/Parquet;
   - Salva o resultado final em `library_1064nm.parquet`.

Por fim, a célula extrai as colunas relevantes para o MatchingPhase: $L_x$, $L_y$, altura $H$, `S_complex_TE` e `S_complex_TM`, formando o `library_df`.

<details>
<summary><b>Show English</b></summary>

**EN** — This cell prepares the **meta-atom library** used in phase matching:

1. Sets the target frequency in GHz (`freq_target_ghz = 281760.0`), equivalent to $\lambda = 1064\ \text{nm}$;
2. Defines the Parquet cache file (`library_1064nm.parquet`) under `data/meta_library/`;
3. If the Parquet file exists:
   - Loads the `DataFrame` into `df_1064`;
   - If complex columns are missing (only real/imag parts), rebuilds `S_complex_TE` and `S_complex_TM` from their real and imaginary components;
4. If no cache is found:
   - Uses `touchstone_to_dataframe` to convert all Touchstone files in `bib_folder` into a single aggregated `DataFrame`;
   - Filters rows with `frequencia_ghz ≈ 281760.0`;
   - Uses `append_derived_columns` to create complex columns `S_complex_TE` (from S13) and `S_complex_TM` (from S24);
   - Splits these complex columns into real/imag before saving to Parquet, avoiding Arrow/Parquet complex-type issues;
   - Saves the final result as `library_1064nm.parquet`.

Finally, the cell selects the relevant columns for MatchingPhase: $L_x$, $L_y$, height $H$, `S_complex_TE` and `S_complex_TM`, forming `library_df`.



In [32]:
freq_target_ghz = 281760.0
cache_path = repo_root / "data" / "meta_library" / "library_1064nm.parquet"

if cache_path.is_file():
    print(f"Carregando biblioteca em cache: {cache_path}")
    df_1064 = pd.read_parquet(cache_path)

    # Se vier sem as colunas complexas (salvas como real/imag), reconstrói:
    if "S_complex_TE" not in df_1064.columns and "S_complex_TE_real" in df_1064.columns:
        df_1064["S_complex_TE"] = (
            df_1064["S_complex_TE_real"].to_numpy()
            + 1j * df_1064["S_complex_TE_imag"].to_numpy()
        )
        df_1064["S_complex_TM"] = (
            df_1064["S_complex_TM_real"].to_numpy()
            + 1j * df_1064["S_complex_TM_imag"].to_numpy()
        )

else:
    print("Nenhuma biblioteca em cache encontrada. Gerando a partir dos Touchstone...")
    df_raw = touchstone_to_dataframe(str(bib_folder), recursive=False)
    df_1064 = df_raw[np.isclose(df_raw["frequencia_ghz"], freq_target_ghz)].copy()

    df_1064 = append_derived_columns(
        df_1064,
        te_cols=("S13_real", "S13_imag"),  # TE = S13
        tm_cols=("S24_real", "S24_imag"),  # TM = S24
        unwrap_phase=False,
        phase_unit="rad"
    )

    cache_path.parent.mkdir(parents=True, exist_ok=True)

    # 👉 NÃO salvar complexos em Parquet: decompõe em real/imag
    df_to_save = df_1064.copy()
    df_to_save["S_complex_TE_real"] = np.real(df_to_save["S_complex_TE"].to_numpy())
    df_to_save["S_complex_TE_imag"] = np.imag(df_to_save["S_complex_TE"].to_numpy())
    df_to_save["S_complex_TM_real"] = np.real(df_to_save["S_complex_TM"].to_numpy())
    df_to_save["S_complex_TM_imag"] = np.imag(df_to_save["S_complex_TM"].to_numpy())

    df_to_save = df_to_save.drop(columns=["S_complex_TE", "S_complex_TM"])

    df_to_save.to_parquet(cache_path)
    print(f"Biblioteca salva em cache em: {cache_path}")

print(f"✔ Biblioteca em 1064 nm carregada: {len(df_1064)} linhas")

# Vamos usar só as colunas relevantes para o MatchingPhase
library_df = df_1064[["L_x", "L_y", "H", "S_complex_TE", "S_complex_TM"]].copy()


Carregando biblioteca em cache: c:\Users\vinicius23011\MATLAB\Projects\TCC\data\meta_library\library_1064nm.parquet
✔ Biblioteca em 1064 nm carregada: 41553 linhas


#### Comentário 

O print mostra:
- Que a biblioteca foi carregada do cache;
- O número de linhas (41553), refletindo a quantidade de meta-átomos disponíveis para o casamento de fase.


### 5. Função principal de casamento de fase (MatchingPhase)  

**PT** — Esta função implementa o núcleo do **algoritmo de casamento de fase** descrito na monografia:

- Entradas:
  - `library_df`: biblioteca filtrada contendo $L_x$, $L_y$, `S_complex_TM`, `S_complex_TE` e opcionalmente `H`;
  - `target_phase_tm`: mapa de fase alvo para TM (holograma, polarização X);
  - `target_phase_te`: mapa de fase alvo para TE (Dammann, polarização Y);
- Parâmetros opcionais:
  - `use_height`, `target_height_um`, `height_tolerance_um`, `height_col`: permitem filtrar a biblioteca por uma faixa de altura (em nm) antes do matching, se desejado.

Para cada pixel $(i,j)$:

1. Constrói os alvos complexos:
   - $E_{\text{TM}} = e^{i\phi_{\text{TM}}(i,j)}$;
   - $E_{\text{TE}} = e^{i\phi_{\text{TE}}(i,j)}$;
2. Calcula os erros quadráticos:
   - $\text{erro}_\text{TM} = |S_\text{TM} - E_\text{TM}|^2$;
   - $\text{erro}_\text{TE} = |S_\text{TE} - E_\text{TE}|^2$;
3. Soma os erros: $\text{erro}_\text{total} = \text{erro}_\text{TM} + \text{erro}_\text{TE}$;
4. Encontra o índice do meta-átomo que minimiza $\text{erro}_\text{total}$;
5. Preenche:
   - `layout_lx[i, j]` e `layout_ly[i, j]` com os valores de $L_x$ e $L_y$ do meta-átomo escolhido;
   - `error_map[i, j]` com o erro mínimo encontrado.

A função retorna:

- `layout_lx`, `layout_ly`: mapas bidimensionais dos semi-eixos da elipse (layout físico da metassuperfície);
- `error_map`: mapa bidimensional do erro RMS por pixel (raiz quadrada do erro total no espaço complexo).

<details>
<summary><b>Show English</b></summary>

**EN** — This function implements the core **phase-matching algorithm** described in the thesis:

- Inputs:
  - `library_df`: filtered library containing $L_x$, $L_y$, `S_complex_TM`, `S_complex_TE` and optionally `H`;
  - `target_phase_tm`: target phase map for TM (hologram, X polarization);
  - `target_phase_te`: target phase map for TE (Dammann, Y polarization);
- Optional parameters:
  - `use_height`, `target_height_um`, `height_tolerance_um`, `height_col`: allow height-based filtering of the library (in nm), if desired.

For each pixel $(i,j)$:

1. Builds complex targets:
   - $E_{\text{TM}} = e^{i\phi_{\text{TM}}(i,j)}$;
   - $E_{\text{TE}} = e^{i\phi_{\text{TE}}(i,j)}$;
2. Computes squared errors:
   - $\text{error}_\text{TM} = |S_\text{TM} - E_\text{TM}|^2$;
   - $\text{error}_\text{TE} = |S_\text{TE} - E_\text{TE}|^2$;
3. Sums errors: $\text{error}_\text{total} = \text{error}_\text{TM} + \text{error}_\text{TE}$;
4. Finds the meta-atom index minimizing $\text{error}_\text{total}$;
5. Fills:
   - `layout_lx[i, j]` and `layout_ly[i, j]` with the chosen meta-atom’s $L_x$ and $L_y$;
   - `error_map[i, j]` with the minimal error.

The function returns:

- `layout_lx`, `layout_ly`: 2D maps of ellipse semi-axes (physical layout of the metasurface);
- `error_map`: 2D map of per-pixel RMS error (square root of total complex error).


In [33]:
def perform_phase_matching(
    library_df: pd.DataFrame,
    target_phase_tm: np.ndarray,
    target_phase_te: np.ndarray,
    use_height: bool = False,
    target_height_um: float = None,
    height_tolerance_um: float = 0.05,
    height_col: str = "H",
):
    """
    Algoritmo de casamento de fase usado nas figuras da monografia.

    - TM (holograma, polarização X)  -> S_complex_TM
    - TE (Dammann, polarização Y)   -> S_complex_TE

    O erro por pixel é a distância euclidiana no espaço complexo:
    sqrt(|S_TM - e^{i φ_TM}|^2 + |S_TE - e^{i φ_TE}|^2).
    """
    df_lib = library_df.copy()

    # Filtrar por altura, se desejado (H na biblioteca está em nm)
    if use_height and target_height_um is not None:
        target_nm = target_height_um * 1e3
        tol_nm = height_tolerance_um * 1e3
        mask = (df_lib[height_col] >= target_nm - tol_nm) & (df_lib[height_col] <= target_nm + tol_nm)
        df_lib = df_lib[mask]
        if df_lib.empty:
            raise ValueError(
                f"Nenhum meta-átomo com {height_col} dentro de "
                f"[{target_nm - tol_nm:.1f}, {target_nm + tol_nm:.1f}] nm"
            )
        print(f"   Biblioteca filtrada por altura: {len(df_lib)} meta-átomos restantes.")

    # Extrai vetores da biblioteca
    lib_lx = df_lib["L_x"].to_numpy()
    lib_ly = df_lib["L_y"].to_numpy()
    lib_S_tm = df_lib["S_complex_TM"].to_numpy(dtype=np.complex128)
    lib_S_te = df_lib["S_complex_TE"].to_numpy(dtype=np.complex128)

    altura, largura = target_phase_tm.shape

    layout_lx = np.zeros_like(target_phase_tm, dtype=float)
    layout_ly = np.zeros_like(target_phase_tm, dtype=float)
    error_map = np.zeros_like(target_phase_tm, dtype=float)

    print("\nIniciando MatchingPhase (TM = holograma X, TE = Dammann Y)...")
    for i in tqdm(range(altura), desc="Progresso MatchingPhase"):
        for j in range(largura):
            target_tm = np.exp(1j * target_phase_tm[i, j])
            target_te = np.exp(1j * target_phase_te[i, j])

            error_tm = np.abs(lib_S_tm - target_tm) ** 2
            error_te = np.abs(lib_S_te - target_te) ** 2
            total_error = error_tm + error_te

            best_idx = np.argmin(total_error)

            layout_lx[i, j] = lib_lx[best_idx]
            layout_ly[i, j] = lib_ly[best_idx]
            error_map[i, j] = total_error[best_idx]

    return layout_lx, layout_ly, np.sqrt(error_map)

### 6. Seleção do último run de GS (canal X / holografia)  

**PT** — Nesta célula:

- Define-se `gs_x_root` apontando para os resultados de GS em X, no experimento `demo_ilum_ufabc`;
- A função `get_latest_subdir(gs_x_root)` escolhe a subpasta mais recente (último run de GS para esse demo);
- Monta-se um dicionário `phase_x_paths` que associa o nome da imagem (`"ilum"`, `"ufabc"`) ao caminho do arquivo `.txt` contendo o mapa de fase GS para polarização X (TM) daquela imagem.

<details>
<summary><b>Show English</b></summary>

**EN** — In this cell:

- `gs_x_root` is set to the X-channel GS results for the `demo_ilum_ufabc` experiment;
- `get_latest_subdir(gs_x_root)` chooses the most recent GS run;
- `phase_x_paths` is built as a dictionary mapping image names (`"ilum"`, `"ufabc"`) to the `.txt` files containing the GS phase map in X (TM) polarization.


In [34]:
gs_x_root = results_holo_root / "gs_x" / "demo_ilum_ufabc"
run_x = get_latest_subdir(gs_x_root)

phase_x_paths = {
    "ilum": run_x / "ilum" / "phase_map__ilum__X__λ_1064nm__z_380um__dx_520nm__iter_300.txt",
    "ufabc": run_x / "ufabc" / "phase_map__ufabc__X__λ_1064nm__z_380um__dx_520nm__iter_300.txt",
    # "hjv": run_x / "hjv" / "phase_map__hjv__X__λ_1064nm__z_380um__dx_520nm__iter_300.txt",
    # "zju": run_x / "zju" / "phase_map__zju__X__λ_1064nm__z_380um__dx_520nm__iter_300.txt",
    # "zju_logo": run_x / "zju_logo" / "phase_map__zju_logo__X__λ_1064nm__z_380um__dx_520nm__iter_300.txt",
}

   Último run em demo_ilum_ufabc: 2025-11-16T00-55-43Z


### 7. Seleção do último run de Dammann (canal Y / TE)  

**PT** — Esta célula:

- Define `gs_y_root` como a pasta de resultados da simulação para a grade de Dammann `demo_dammannY`;
- Usa `get_latest_subdir(gs_y_root)` para selecionar automaticamente o último run;
- Monta o caminho do arquivo de fase TE tilado no plano da máscara (`full_phase_tile__Y__...txt`);
- Carrega esse mapa de fase em `phase_map_y` usando `load_phase_txt` e imprime seu shape (e.g. `(450, 450)`).

<details>
<summary><b>Show English</b></summary>

**EN** — This cell:

- Sets `gs_y_root` to the Dammann simulation results for `demo_dammannY`;
- Uses `get_latest_subdir(gs_y_root)` to automatically select the latest run;
- Builds the path to the tiled TE phase file on the mask plane (`full_phase_tile__Y__...txt`);
- Loads the phase map into `phase_map_y` via `load_phase_txt` and prints its shape (e.g. `(450, 450)`).



In [35]:
gs_y_root = results_holo_root / "gs_y" / "demo_dammannY"
run_y = get_latest_subdir(gs_y_root)

phase_y_file = (
    run_y
    / "full_phase_tile__Y__λ_1064nm__P_520nm__scpix_45px__nsc_10__iter_500__seed_0.txt"
)
phase_map_y = load_phase_txt(phase_y_file)
print("Mapa de fase Y (Dammann) carregado:", phase_map_y.shape)

   Último run em demo_dammannY: 2025-11-16T01-14-26Z
Mapa de fase Y (Dammann) carregado: (450, 450)


### 8. Parâmetros ópticos e de execução | Optical and Execution Parameters  

#### **PT - Descrição**  
Os parâmetros abaixo definem a geometria da propagação numérica e são consistentes com a etapa de GS descrita na monografia:

| Parâmetro | Significado                                       | Valor        |
|----------|----------------------------------------------------|-------------|
| `wavelength` | Comprimento de onda $\lambda$                   | $1064 \,\text{nm}$ |
| `z_m`        | Distância entre plano da máscara e plano da imagem | $380 \,\mu\text{m}$ |
| `dx_m`       | Passo de amostragem espacial no plano da máscara   | $520 \,\text{nm}$ |
| `NA`         | Abertura numérica equivalente da lente            | 0.65        |

Esses valores são usados pelo módulo `gs_asm` para reconstruir numericamente o campo no plano da imagem.

<details>
<summary><b>EN - Description</b></summary>

These parameters define the geometry of the numerical propagation and are consistent with the GS step described in the thesis:

| Parameter  | Meaning                                             | Value       |
|-----------|------------------------------------------------------|-------------|
| `wavelength` | Wavelength $\lambda$                              | $1064 \,\text{nm}$ |
| `z_m`        | Distance between mask plane and image plane       | $380 \,\mu\text{m}$ |
| `dx_m`       | Spatial sampling pitch at the mask plane          | $520 \,\text{nm}$ |
| `NA`         | Equivalent lens numerical aperture                | 0.65        |

These values are used by the `gs_asm` module to numerically reconstruct the field at the image plane.

</details>

In [36]:
# Parâmetros físicos do sistema
wavelength = 1064e-9  # 1064 nm
z_m = 380e-6          # 380 μm
dx_m = 520e-9         # pixel pitch
NA = 0.65

### 9. 📂 Criação da pasta de saída para este run de MatchingPhase  

**PT** — Esta célula organiza os resultados em uma hierarquia de pastas baseada em timestamp:

- Define `demo_name = "demo_ilum_ufabc"`, identificando o conjunto de alvos (ilum + ufabc);
- Gera um `run_id` usando `datetime.utcnow()` no formato `YYYY-MM-DDTHH-MM-SSZ`;
- Cria `base_output_dir = results_pm_root / demo_name / run_id`;
- Garante que o diretório existe (`mkdir(parents=True, exist_ok=True)`);
- Imprime o caminho onde todos os arquivos desse run serão salvos.

<details>
<summary><b>Show English</b></summary>

**EN** — This cell organizes the results into a timestamp-based directory hierarchy:

- Sets `demo_name = "demo_ilum_ufabc"` as an identifier for the target set (ilum + ufabc);
- Generates a `run_id` via `datetime.utcnow()` in `YYYY-MM-DDTHH-MM-SSZ` format;
- Creates `base_output_dir = results_pm_root / demo_name / run_id`;
- Ensures the directory exists (`mkdir(parents=True, exist_ok=True)`);
- Prints the full path where all files from this run will be stored.



In [37]:
from datetime import datetime

demo_name = "demo_ilum_ufabc"

run_id = datetime.utcnow().strftime("%Y-%m-%dT%H-%M-%SZ")
base_output_dir = results_pm_root / demo_name / run_id
base_output_dir.mkdir(parents=True, exist_ok=True)

print(f"\n📁 Resultados do MatchingPhase serão salvos em: {base_output_dir}")



📁 Resultados do MatchingPhase serão salvos em: c:\Users\vinicius23011\MATLAB\Projects\TCC\results\meta_library\phase_matching\demo_ilum_ufabc\2025-11-16T03-37-41Z


### 10. Função de alto nível: MatchingPhase + reconstrução + métricas  

**PT** — Esta função reúne todo o pipeline para uma única imagem (`img_name`):

1. **Carregamento da fase X (TM/holograma)**  
   - Lê o arquivo `.txt` do mapa de fase GS para o alvo corrente;
   - Imprime o shape do mapa de fase (e.g. `(450, 450)`).

2. **Verificação de compatibilidade com o mapa TE**  
   - Usa `ensure_same_shape(phase_map_x, phase_map_y)` para garantir que TM e TE têm o mesmo tamanho.

3. **Execução do MatchingPhase (casamento de fase)**  
   - Chama `perform_phase_matching` com:
     - `target_phase_tm = phase_map_x`;
     - `target_phase_te = phase_map_y`;
   - Obtém `layout_lx`, `layout_ly`, `error_map`;
   - Imprime shape das matrizes e as estatísticas de erro RMS médio e máximo.

4. **Salvar resultados específicos da imagem**  
   - Cria `out_dir = base_output_dir / img_name`;
   - Salva:
     - `layout_lx`, `layout_ly`, `error_map` em `.npy`;
     - `layout_lx`, `layout_ly` em `.csv` (sem cabeçalho) para uso em fabricação.

5. **Visualização de layouts e mapa de erro**  
   - Gera uma figura com três painéis:
     - Mapa de $L_x$;
     - Mapa de $L_y$;
     - Mapa de erro RMS;
   - Salva como `layouts_and_error__<img>.png`.

6. **Histograma dos erros**  
   - Gera o histograma da distribuição de `error_map.flatten()` com número de bins fixo;
   - Salva como `error_histogram__<img>.png`.

7. **Reconstrução final do holograma (TM)**  
   - Faz `groupby(["L_x", "L_y"])` na biblioteca para ter um único valor de `S_complex_TM` por geometria, usando a média;
   - Reconstrói o campo complexo TM final no plano da máscara usando o layout (`MultiIndex` + `reindex`);
   - Converte para fase (`phase_final_tm`) e chama `GSx.reconstruct_image` com os parâmetros ópticos definidos anteriormente;
   - Carrega a imagem-alvo bruta (`<img>.png` em `targets_dir`) e a pré-processa para ter o mesmo tamanho da reconstruída;
   - Plota alvo vs reconstruída lado a lado, salvando em `reconstruction__<img>.png`.

8. **Cálculo da correlação de Pearson**  
   - Usa `GSx.calculate_correlation(target_image, recon_image)` para obter a correlação de Pearson;
   - Imprime o valor e salva em `metrics__<img>.txt`, junto com:
     - `Erro_RMS_medio`;
     - `Erro_RMS_max`.

<details>
<summary><b>Show English</b></summary>

**EN** — This function ties together the entire pipeline for a single image (`img_name`):

1. **Load X-phase (TM/hologram)**  
   - Reads the `.txt` GS phase map for the current target;
   - Prints the phase map shape (e.g. `(450, 450)`).

2. **Shape consistency with TE map**  
   - Uses `ensure_same_shape(phase_map_x, phase_map_y)` to ensure TM and TE have the same size.

3. **Run MatchingPhase (phase matching)**  
   - Calls `perform_phase_matching` with:
     - `target_phase_tm = phase_map_x`;
     - `target_phase_te = phase_map_y`;
   - Obtains `layout_lx`, `layout_ly`, `error_map`;
   - Prints shapes and RMS error statistics (mean and max).

4. **Save per-image results**  
   - Creates `out_dir = base_output_dir / img_name`;
   - Saves:
     - `layout_lx`, `layout_ly`, `error_map` as `.npy`;
     - `layout_lx`, `layout_ly` as `.csv` (no header) for fabrication workflows.

5. **Layout and error visualization**  
   - Produces a three-panel figure:
     - $L_x$ map;
     - $L_y$ map;
     - RMS error map;
   - Saves it as `layouts_and_error__<img>.png`.

6. **Error histogram**  
   - Plots the histogram of `error_map.flatten()` with a fixed number of bins;
   - Saves it as `error_histogram__<img>.png`.

7. **Final hologram reconstruction (TM)**  
   - Groups the library by `["L_x", "L_y"]` to obtain a single `S_complex_TM` per geometry (mean value);
   - Reconstructs the final TM complex field on the mask plane using the layout (`MultiIndex` + `reindex`);
   - Converts to phase (`phase_final_tm`) and calls `GSx.reconstruct_image` with the optical parameters defined earlier;
   - Loads the raw target image (`<img>.png` in `targets_dir`) and preprocesses it to match the reconstructed size;
   - Plots target vs reconstructed side by side, saving as `reconstruction__<img>.png`.

8. **Pearson correlation computation**  
   - Uses `GSx.calculate_correlation(target_image, recon_image)` to compute the Pearson correlation;
   - Prints the value and writes it to `metrics__<img>.txt`, along with:
     - `Erro_RMS_medio`;
     - `Erro_RMS_max`.

In [38]:
def run_matching_for_image(img_name, phase_x_file):
    """
    Executa todo o pipeline de MatchingPhase + reconstrução
    para uma única imagem (img_name).
    Usa:
      - phase_map_y       (fase Dammann, TE)
      - library_df        (biblioteca de meta-átomos)
      - base_output_dir   (raiz deste run)
      - wavelength, z_m, dx_m, NA
      - GSx               (módulo gs_asm importado como GSx)
    """
    print(f"\n=== Imagem: {img_name} ===")
    print("Carregando mapa de fase X (holograma)...")
    phase_map_x = load_phase_txt(phase_x_file)
    print("   Shape X:", phase_map_x.shape)

    # Garantir shapes compatíveis entre X (TM) e Y (TE)
    phase_tm, phase_te = ensure_same_shape(phase_map_x, phase_map_y)

    # 6.1) MatchingPhase propriamente dito
    layout_lx, layout_ly, error_map = perform_phase_matching(
        library_df=library_df,
        target_phase_tm=phase_tm,   # TM -> holograma X
        target_phase_te=phase_te,   # TE -> Dammann Y
        use_height=False,           # altura fixa em 600 nm na Bib1-27x27-perdas
        target_height_um=0.6,
        height_tolerance_um=0.05,
        height_col="H",
    )

    print(f"   Layout L_x shape: {layout_lx.shape}")
    print(f"   Layout L_y shape: {layout_ly.shape}")
    print(f"   Mapa de erro shape: {error_map.shape}")
    print(f"   Erro RMS médio: {np.mean(error_map):.4f}")
    print(f"   Erro RMS máximo: {np.max(error_map):.4f}")

    # 6.2) Salvar resultados específicos da imagem
    out_dir = base_output_dir / img_name
    out_dir.mkdir(parents=True, exist_ok=True)

    # Arrays
    np.save(out_dir / f"layout_lx__{img_name}.npy", layout_lx)
    np.save(out_dir / f"layout_ly__{img_name}.npy", layout_ly)
    np.save(out_dir / f"error_map__{img_name}.npy", error_map)

    # CSVs de layout para fabricação
    pd.DataFrame(layout_lx).to_csv(
        out_dir / f"layout_lx__{img_name}.csv",
        index=False,
        header=False,
    )
    pd.DataFrame(layout_ly).to_csv(
        out_dir / f"layout_ly__{img_name}.csv",
        index=False,
        header=False,
    )

    # 6.3) Figuras: layouts + mapa de erro
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    fig.suptitle(f"Phase Matching – {img_name}", fontsize=14)

    im1 = axes[0].imshow(layout_lx, cmap="viridis")
    axes[0].set_title("Layout L_x (nm)")
    axes[0].axis("off")
    plt.colorbar(im1, ax=axes[0])

    im2 = axes[1].imshow(layout_ly, cmap="viridis")
    axes[1].set_title("Layout L_y (nm)")
    axes[1].axis("off")
    plt.colorbar(im2, ax=axes[1])

    im3 = axes[2].imshow(error_map, cmap="hot")
    axes[2].set_title("Erro RMS (|S_TM,TE - alvo|)")
    axes[2].axis("off")
    plt.colorbar(im3, ax=axes[2])

    plt.tight_layout()
    fig.savefig(
        out_dir / f"layouts_and_error__{img_name}.png",
        dpi=150,
        bbox_inches="tight",
    )
    plt.close(fig)

    # Histograma de erros
    plt.figure(figsize=(6, 4))
    plt.hist(error_map.flatten(), bins=50, edgecolor="black")
    plt.xlabel("Erro RMS")
    plt.ylabel("Contagem")
    plt.title(f"Distribuição de Erros – {img_name}")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    plt.savefig(
        out_dir / f"error_histogram__{img_name}.png",
        dpi=150,
        bbox_inches="tight",
    )
    plt.close()

    # 6.4) Reconstrução final do holograma (TM) e correlação, usando gs_asm
    print("Reconstruindo imagem final (TM, holograma X) para validação...")

    # 🔹 Primeiro, garantimos que cada par (L_x, L_y) apareça só uma vez
    #    (se houver várias alturas ou duplicatas, fazemos a média de S_complex_TM)
    lib_tm_unique = (
        library_df
        .groupby(["L_x", "L_y"], as_index=False)["S_complex_TM"]
        .mean()
    )

    lookup_tm = lib_tm_unique.set_index(["L_x", "L_y"])["S_complex_TM"]

    # Reconstruir campo complexo TM a partir do layout (usando MultiIndex único)
    index = pd.MultiIndex.from_arrays(
        [layout_lx.ravel(), layout_ly.ravel()],
        names=["L_x", "L_y"],
    )
    final_field_tm_flat = lookup_tm.reindex(index).to_numpy()

    # Só por segurança, checar se sobrou algum NaN (não deveria)
    if np.isnan(final_field_tm_flat).any():
        n_nans = np.isnan(final_field_tm_flat).sum()
        print(f"[AVISO] {n_nans} posições do layout não encontradas na biblioteca após o groupby.")
        # Se quiser, pode fazer:
        # final_field_tm_flat = np.nan_to_num(final_field_tm_flat, nan=0+0j)

    final_field_tm = final_field_tm_flat.reshape(layout_lx.shape)

    # Mapa de fase final TM
    phase_final_tm = np.angle(final_field_tm)

    # Reconstrução ótica no plano da imagem
    recon_image = GSx.reconstruct_image(phase_final_tm, wavelength, z_m, dx_m, NA)

    # Carregar a IMAGEM-ALVO BRUTA (sem eixos), ex: ilum.png, ufabc.png
    target_png_path = targets_dir / f"{img_name}.png"

    # Garante que o target tenha EXATAMENTE o mesmo tamanho da reconstruída
    h, w = recon_image.shape
    target_image = GSx.load_and_preprocess_image(
        str(target_png_path),
        target_size=(w, h),   # normalmente (450, 450)
    )

    # Plot comparativo alvo vs reconstruída
    plt.figure(figsize=(10, 4))
    plt.subplot(1, 2, 1)
    plt.imshow(target_image, cmap="gray")
    plt.title(f"Imagem Alvo – {img_name}")
    plt.axis("off")

    plt.subplot(1, 2, 2)
    plt.imshow(recon_image, cmap="gray")
    plt.title(f"Imagem Reconstruída – {img_name}")
    plt.axis("off")

    plt.tight_layout()
    plt.savefig(
        out_dir / f"reconstruction__{img_name}.png",
        dpi=150,
        bbox_inches="tight",
    )
    plt.close()

    # Agora as duas têm o mesmo shape → correlação bem definida
    corr = GSx.calculate_correlation(target_image, recon_image)
    print(f"   Correlação de Pearson final ({img_name}): {corr:.4f}")

    # Salvar métricas em um txt simples
    with open(out_dir / f"metrics__{img_name}.txt", "w", encoding="utf-8") as f:
        f.write(f"Erro_RMS_medio = {np.mean(error_map):.6f}\n")
        f.write(f"Erro_RMS_max   = {np.max(error_map):.6f}\n")
        f.write(f"Pearson_final  = {corr:.6f}\n")


### 11. Loop principal: executando o MatchingPhase para todas as imagens  

**PT** — Esta célula é o *driver* do notebook para o conjunto de imagens configuradas:

- Percorre o dicionário `phase_x_paths`;
- Para cada par `(img_name, phase_x_file)`, chama `run_matching_for_image(img_name, phase_x_file)`;
- Ao final, imprime:
  - Uma mensagem de conclusão;
  - O caminho de saída `base_output_dir`.

<details>
<summary><b>Show English</b></summary>

**EN** — This cell is the notebook’s *driver* for the configured set of images:

- Iterates over the `phase_x_paths` dictionary;
- For each `(img_name, phase_x_file)` pair, calls `run_matching_for_image(img_name, phase_x_file)`;
- At the end, prints:
  - A completion message;
  - The output directory `base_output_dir`.


In [39]:
for img_name, phase_x_file in phase_x_paths.items():
    run_matching_for_image(img_name, phase_x_file)

print("\n✅ MatchingPhase concluído para todas as imagens configuradas.")
print("Resultados em:", base_output_dir)



=== Imagem: ilum ===
Carregando mapa de fase X (holograma)...
   Shape X: (450, 450)

Iniciando MatchingPhase (TM = holograma X, TE = Dammann Y)...


Progresso MatchingPhase: 100%|██████████| 450/450 [01:19<00:00,  5.63it/s]


   Layout L_x shape: (450, 450)
   Layout L_y shape: (450, 450)
   Mapa de erro shape: (450, 450)
   Erro RMS médio: 0.1513
   Erro RMS máximo: 0.4152
Reconstruindo imagem final (TM, holograma X) para validação...
   Correlação de Pearson final (ilum): 0.7381

=== Imagem: ufabc ===
Carregando mapa de fase X (holograma)...
   Shape X: (450, 450)

Iniciando MatchingPhase (TM = holograma X, TE = Dammann Y)...


Progresso MatchingPhase: 100%|██████████| 450/450 [01:12<00:00,  6.22it/s]


   Layout L_x shape: (450, 450)
   Layout L_y shape: (450, 450)
   Mapa de erro shape: (450, 450)
   Erro RMS médio: 0.1509
   Erro RMS máximo: 0.4148
Reconstruindo imagem final (TM, holograma X) para validação...
   Correlação de Pearson final (ufabc): 0.7932

✅ MatchingPhase concluído para todas as imagens configuradas.
Resultados em: c:\Users\vinicius23011\MATLAB\Projects\TCC\results\meta_library\phase_matching\demo_ilum_ufabc\2025-11-16T03-37-41Z


#### Comentário
O log mostra, para cada imagem (por exemplo, `ilum` e `ufabc`):
- Shapes dos layouts e do mapa de erro;
- Erro RMS médio (≈ 0.15) e máximo (≈ 0.41);
- Correlação de Pearson final (≈ 0.738 para ilum e ≈ 0.793 para ufabc);
- Mensagem final confirmando que o MatchingPhase foi concluído para todas as imagens e o diretório de resultados.

### 12. Cálculo de DE e RMSE TE pós-MatchingPhase  

**PT** — As duas últimas células do notebook estendem a análise para o canal TE, reutilizando exatamente a mesma infraestrutura usada para avaliar as grades de Dammann:

1. **Import do módulo `dammann_fft` (célula 12)**  
   - Importa o módulo responsável pelos cálculos de ordens de difração e intensidades em campo distante (`calculate_diffraction_orders`);
   - Tenta `from dammann import dammann_fft as dY`; se não estiver nesse formato, faz fallback para `import dammann_fft as dY`;
   - Imprime:
     - O caminho do arquivo fonte carregado;
     - Os parâmetros de referência: período `P`, comprimento de onda `wavelength`, número de pixels por supercélula (`supercell_pixels`) e número de supercélulas (`n_supercells`).

2. **Loop de métricas TE por imagem (célula 13)**  
   - Faz um `groupby(["L_x", "L_y"])` em `library_df` para garantir um único valor de `S_complex_TE` por par geométrico, usando a média;
   - Para cada alvo já passado pelo MatchingPhase:
     - Carrega os layouts finais `layout_lx` e `layout_ly` associados àquela imagem;
     - Usa um `MultiIndex` baseado em $(L_x, L_y)$ para reindexar `lookup_te` e reconstruir o campo complexo TE final no plano da máscara;
     - Calcula a fase final TE `full_phase_te = angle(full_field_te)`;
     - Chama `dY.calculate_diffraction_orders(...)` usando os mesmos parâmetros `P`, `wavelength`, `supercell_pixels` e `n_supercells` utilizados na simulação original da grade de Dammann;
     - Obtém a lista de intensidades por ordem e o mapa de intensidade em campo distante `I_far`;
     - Calcula:
       - **DE (Diffraction Efficiency)** como $\text{DE} = \frac{\sum \text{intensities}}{\sum I_{\text{far}}}$, isto é, a fração de energia contida nas ordens-alvo em relação à energia total no campo distante;
       - **RMSE** de uniformidade entre as intensidades normalizadas pela média e a distribuição idealmente uniforme ($=1$), via $\text{RMSE} = \sqrt{\frac{1}{N}\sum_k \left(\frac{I_k}{\langle I \rangle} - 1\right)^2}$
     - Imprime `DE` e `RMSE` para cada alvo (ilum, ufabc);
     - Acrescenta `DE_TE` e `RMSE_TE` ao arquivo de métricas da imagem (`metrics__<img>.txt`).

<details>
<summary><b>Show English</b></summary>

**EN** — The last two cells extend the analysis to the TE channel, reusing exactly the same infrastructure used to evaluate the Dammann gratings:

1. **Importing `dammann_fft` module (cell 12)**  
   - Imports the module responsible for diffraction order calculations and far-field intensity (`calculate_diffraction_orders`);
   - Tries `from dammann import dammann_fft as dY`; if that fails, falls back to `import dammann_fft as dY`;
   - Prints:
     - The source file path of the loaded module;
     - The reference parameters: period `P`, wavelength `wavelength`, number of pixels per supercell (`supercell_pixels`), and number of supercells (`n_supercells`).

2. **Per-image TE metrics loop (cell 13)**  
   - Performs a `groupby(["L_x", "L_y"])` on `library_df` to ensure a single `S_complex_TE` per geometry pair, using the mean;
   - For each target already processed by MatchingPhase:
     - Loads the final layouts `layout_lx` and `layout_ly` for that image;
     - Uses a `MultiIndex` based on $(L_x, L_y)$ to reindex `lookup_te` and reconstruct the final TE complex field at the mask plane;
     - Computes the final TE phase `full_phase_te = angle(full_field_te)`;
     - Calls `dY.calculate_diffraction_orders(...)` with the same parameters `P`, `wavelength`, `supercell_pixels`, and `n_supercells` used in the original Dammann simulation;
     - Obtains the list of order intensities and the far-field intensity map `I_far`;
     - Computes:
       - **DE (Diffraction Efficiency)** as the fraction of energy in the selected orders relative to the total energy in the far field;
       - **RMSE** as the uniformity error of the normalized order intensities with respect to the ideal uniform case;
     - Prints `DE` and `RMSE` for each target (ilum, ufabc);
     - Appends `DE_TE` and `RMSE_TE` to each image’s metrics file (`metrics__<img>.txt`).

In [40]:
# === Métricas TE pós-MatchingPhase (DE e RMSE por imagem) ===
# Importa a rotina de ordens de difração do módulo Dammann

try:
    from dammann import dammann_fft as dY
except ModuleNotFoundError:
    # fallback: módulo direto em src/
    import dammann_fft as dY

print("Módulo dammann_fft carregado a partir de:", dY.__file__)
print("Parâmetros Dammann de referência:")
print(f"  P                 = {dY.P} m")
print(f"  wavelength        = {dY.wavelength} m")
print(f"  supercell_pixels  = {dY.supercell_pixels}")
print(f"  n_supercells      = {dY.n_supercells}")


Módulo dammann_fft carregado a partir de: C:\Users\vinicius23011\MATLAB\Projects\TCC\src\dammann\dammann_fft.py
Parâmetros Dammann de referência:
  P                 = 5.2e-07 m
  wavelength        = 1.064e-06 m
  supercell_pixels  = 45
  n_supercells      = 10


In [41]:
# === Cálculo de DE e RMSE no canal TE, pós-MatchingPhase, para cada imagem ===

# 1) Pré-processar a biblioteca: garantir um único valor de S_complex_TE por (L_x, L_y)
lib_te_unique = (
    library_df
    .groupby(["L_x", "L_y"], as_index=False)["S_complex_TE"]
    .mean()
)
lookup_te = lib_te_unique.set_index(["L_x", "L_y"])["S_complex_TE"]

# 2) Loop sobre as imagens que já foram processadas pelo MatchingPhase
for img_name in phase_x_paths.keys():
    out_dir = base_output_dir / img_name

    print(f"\n=== Métricas TE pós-MatchingPhase – {img_name} ===")

    # 2.1) Carregar layouts finais L_x, L_y dessa imagem
    layout_lx = np.load(out_dir / f"layout_lx__{img_name}.npy")
    layout_ly = np.load(out_dir / f"layout_ly__{img_name}.npy")

    # 2.2) Reconstruir campo complexo TE a partir do layout
    index_te = pd.MultiIndex.from_arrays(
        [layout_lx.ravel(), layout_ly.ravel()],
        names=["L_x", "L_y"],
    )

    full_field_te_flat = lookup_te.reindex(index_te).to_numpy()

    # Segurança: tratar eventuais NaNs (caso algum (L_x, L_y) não esteja na biblioteca)
    if np.isnan(full_field_te_flat).any():
        n_nans = np.isnan(full_field_te_flat).sum()
        print(f"[AVISO] {n_nans} posições do layout não encontradas na biblioteca TE; substituindo por 0.")
        full_field_te_flat = np.nan_to_num(full_field_te_flat, nan=0+0j)

    full_field_te = full_field_te_flat.reshape(layout_lx.shape)

    # 2.3) Fase TE final da metassuperfície
    full_phase_te = np.angle(full_field_te)

    # 2.4) Cálculo das ordens de difração e intensidades usando a MESMA rotina do Dammann
    orders, intensities, I_far = dY.calculate_diffraction_orders(
        full_phase_te,
        P=dY.P,
        wavelength=dY.wavelength,
        supercell_pixels=dY.supercell_pixels,
        n_supercells=dY.n_supercells,
    )

    intensities = np.array(intensities)
    total_energy = I_far.sum()

    if total_energy > 0 and len(intensities) > 0:
        de = intensities.sum() / total_energy
        mean_I = intensities.mean()
        rmse = np.sqrt(np.mean((intensities / mean_I - 1.0)**2))
    else:
        de = np.nan
        rmse = np.nan
        print("[AVISO] Não foi possível calcular DE/RMSE (energia total ou número de ordens inválido).")

    print(f"   DE   (TE, pós-MP) = {de:.4f}")
    print(f"   RMSE (TE, pós-MP) = {rmse:.4f}")

    # 2.5) Registrar no mesmo arquivo de métricas da imagem
    with open(out_dir / f"metrics__{img_name}.txt", "a", encoding="utf-8") as f:
        f.write(f"DE_TE   = {de:.6f}\n")
        f.write(f"RMSE_TE = {rmse:.6f}\n")



=== Métricas TE pós-MatchingPhase – ilum ===
   DE   (TE, pós-MP) = 0.9865
   RMSE (TE, pós-MP) = 0.0760

=== Métricas TE pós-MatchingPhase – ufabc ===
   DE   (TE, pós-MP) = 0.9865
   RMSE (TE, pós-MP) = 0.0760


#### Comentário 

O resultado mostrado no log é:
- Para `ilum`:
  - `DE   (TE, pós-MP) = 0.9865`
  - `RMSE (TE, pós-MP) = 0.0760`
- Para `ufabc`:
  - `DE   (TE, pós-MP) = 0.9865`
  - `RMSE (TE, pós-MP) = 0.0760`

O fato de DE e RMSE serem idênticos para as duas imagens reflete que:

- A distribuição angular das ordens de difração após o MatchingPhase é muito similar em ambos os casos;
- O desempenho global da metassuperfície no canal TE (em termos de eficiência e uniformidade da nuvem de pontos) é praticamente o mesmo, o que reforça a ideia de que o algoritmo entrega uma performance estável independentemente do alvo holográfico escolhido no canal TM.